# Get user info from a Discourse instance

Use the Discourse API to download some fun information to display in a bubble chart.

In [ ]:
import requests
import json
from urllib.parse import urljoin
from time import sleep

# You'll need an API Key;
# Mine is stored in a .gitignored file.
with open("api_key.txt", "r") as f:
    line = f.read().strip()
API_KEY = line.split('"')[1]

DISCOURSE_URL = "https://discourse.marksmath.org" # or your site
API_USERNAME = "mark"  # or your username

In [ ]:
def _headers():
    return {
        "Api-Key": API_KEY,
        "Api-Username": API_USERNAME,
    }

def _avatar_url(base_url: str, avatar_template: str, size: int = 120) -> str:
    """
    Discourse returns an avatar_template like:
      "/user_avatar/discourse.example.com/mark/{size}/123_2.png"
    Replace {size} and join with base URL.
    """
    if not avatar_template:
        return ""
    return urljoin(base_url, avatar_template.replace("{size}", str(size)))

def fetch_user_email(base_url, username):
    email_url = f"{base_url}/u/{username}/emails.json"
    r = requests.get(email_url, headers=_headers(), timeout=15)
    if r.status_code == 200:
        payload = r.json()
        # Usually looks like: {'email': 'email@example.com'}
        return payload.get('email')
    return None

def fetch_user_directory(
    base_url: str,
    order: str = "likes_received",
    period: str = "all",   # typical values: 'all', 'yearly', 'quarterly', 'monthly', 'weekly', 'daily'
    limit: int = 50,
    avatar_size: int = 120,
) -> list[dict]:
    """
    Fetch up to `limit` users from the Discourse user directory, ordered by `order`.
    Returns a list of dicts with username, name, activity metrics, and avatar_url.
    Requires the 'user directory' feature to be enabled on the Discourse site.
    """
    collected = []
    page = 0

    while len(collected) < limit:
        url = f"{base_url}/directory_items.json"
        params = {
            "period": period,
            "order": order,
            "page": page,
        }
        r = requests.get(url, headers=_headers(), params=params, timeout=30)
        if r.status_code == 404:
            raise RuntimeError(
                "User directory not enabled or not visible to this API key. "
            )
        r.raise_for_status()
        payload = r.json()

        items = payload.get("directory_items", [])
        if not items:
            break  # no more results

        for it in items:
            try:
                # We're going to grab each user's email every step through this loop
                # so that we can ensure that the username is the first part of their
                # email address.
                # Set sleep just larger than 1 to avoid the 60 API requests per 
                # minute rule.
                sleep(1.1)
                u = it.get("user", {}) or {}
                username = u.get("username")
                email = fetch_user_email(base_url, username)
                if email.split("@")[0] == username:
                    print([username, ' IS the EMAIL'])
                    username = "Anonymous user"
                row = {
                    "username": username,
                    "user_id": u.get("id"),
                    "likes_received": it.get("likes_received"),
                    # "likes_given": it.get("likes_given"),
                    # "post_count": it.get("post_count"),
                    # "topic_count": it.get("topic_count"),
                    # "time_read": it.get("time_read"),
                    # "days_visited": it.get("days_visited"),
                    # "posts_read": it.get("posts_read"),
                    # "recent_time_read": it.get("recent_time_read"),
                    # "recent_posts_read": it.get("recent_posts_read"),
                    "avatar_url": _avatar_url(base_url, u.get("avatar_template"), size=avatar_size)
                }
                collected.append(row)
                if len(collected) >= limit:
                    break
            except Exception as e:
                print(e)
        page += 1

    return collected[:limit]

In [ ]:
# Example: top 80 users by likes received, across all time
users = fetch_user_directory(
    DISCOURSE_URL,
    order="likes_received",
    period="all",
    limit=80,
    avatar_size=120,
)

In [ ]:
len(users)

In [ ]:
with open("users_with_likes.json", "w") as f:
    json.dump(users, f)